scaricare hdf view richiede acco gratuito per i download

+ link manuale prisma obsoleti? https://wiki.earthdata.nasa.gov/display/DAS/Toolkit+Downloads  

+ **prisma usa librerie hdf-eos5 v1.13, hdf-eos5 è ora a versione 1.16**  

+ esempio di mmap su hdf5(non hdf-eos5) con julia https://github.com/JuliaIO/HDF5.jl/blob/master/test/mmap.jl  

+ immagine docker per hdf eos lib https://hub.docker.com/r/hdfeos/hdfeos  

+ sdp toolkit: https://wiki.earthdata.nasa.gov/display/DAS/SDP+Toolkit+and+HDF-EOS+EOSDIS++Core+System+Project  

+ tutorial (in c) https://portal.hdfgroup.org/display/HDF5/Learning+the+Basics

+ hdfview: tool da https://support.hdfgroup.org/ftp/HDF5/releases/HDF-JAVA/hdf-java-2.10.1/bin/ (plugin non sembra funzionare con versione più recente)
  plugin per hdf-eos da https://wiki.earthdata.nasa.gov/display/DAS/HDFView+HDF-EOS+Plug-in  
  guida installazione: https://newsroom.gsfc.nasa.gov/sdptoolkit/HDFView/README_HDFView_plugin_v2.10.html  
  
+ documentazione pkg hdf5 https://juliaio.github.io/HDF5.jl/stable/

In [35]:
# docu package hdf5 per julia https://juliaio.github.io/HDF5.jl/stable/
# using Pkg
# pkg.add("HDF5")
using HDF5

#errori posson lasciare aperti file-> file inseriti in struct dinamica per poi esser facilmente chiusi tutti
openfiles = Dict()
counter = 0
# funzione che chiude tutti i file aperti
function closeall(dict)
    for (key, value) in dict
        close(pop!(dict, key))    
    end
    counter = 0
    openfiles
end

closeall (generic function with 2 methods)

apertura file con h5open(filename::string, mode::string) -> file   
mode:  

| mode | Meaning |
|---|---|
|"r"|read-only|
|"r+"|read-write, preserving any existing contents|
|"cw"|read-write, create file if not existing, preserve existing contents|
|"w"|read-write, destroying any existing contents (if any)|

In [3]:
testfilename = "test.h5"
fid = h5open(testfilename, "w") # crea file hdf5 vuoto (o lo apre e basta se già esiste)

close(fid)


HDF5 data file: test.h5

In [6]:
close(fid)
rm(testfilename) #deleta file di prova

In [4]:
isopen(fid)

true

In [18]:
function foo(x) 
    println(typeof(x))
end

foo (generic function with 1 method)

In [36]:
#apro in lettura un file sample preso da hdfviewer, faccio qualcosa, chiudo file

file2 = h5open("hdf5_test.h5", "r")
foo(file2)
openfiles["file $counter"]=file2
counter = counter+1;
closeall(openfiles)
isopen(file2)
openfiles

HDF5File


Dict{Any,Any}()

closeall (generic function with 1 method)

In [37]:
openfiles

Dict{Any,Any}()